# PBAs: E Above Hull Analysis

In this notebook, I conduct an analysis of the energy above hull data for the PBAs. In materials science, energy above hull is a measure of a chemical structure's stability, where a lower energy above hull is more stable. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Pre-processing and Loading

Before we can load the json into python, we need to change a few formatting issues with the file:

In [2]:
# with open('pba.json', 'r') as file :
#   pba_json = file.read()

In [3]:
# #Getting rid of the /* i */ and replacing with a comma:
# for i in range(1,700):
#     j = str(i)
#     pba_json = pba_json.replace('/* ' + j + ' */', ',')

# print(pba_json[:100])

In [4]:
# #Adding square brackets:
# pba_json = '[\n' + pba_json + '\n]'
# #Deleting first comma:
# pba_json = pba_json[:2] + pba_json[3:]

In [5]:
# #Getting rid of the ObjectId() tag from the _id value:
# pba_json = pba_json.replace("ObjectId(", "")
# pba_json = pba_json.replace(")", "")

In [6]:
# #Saving to file as pba_1.json
# pba_1 = open('pba_1.json', 'w')
# pba_1.write(pba_json)
# pba_1.close()

Now we are ready to load the file using monty.serialization.loadfn, which loads the json into a list of dictionary entries.

In [7]:
from monty.serialization import loadfn

In [8]:
data_1 = loadfn('pba_1.json')

In [9]:
len(data_1)

536

In [10]:
data_1[1]

{'_id': '58e5d103d95cbb63a64878f0', 'input': {'structure': Structure Summary
  Lattice
      abc : 9.9509025313318 9.9509025313318 9.9509025313318
   angles : 89.99613296435679 90.00386703564321 90.00386703564321
   volume : 985.3429511575596
        A : 9.95090252 -0.0003358 -0.0003358
        B : -0.0003358 9.95090252 0.0003358
        C : -0.0003358 0.0003358 9.95090252
  PeriodicSite: Ca (7.4762, 7.4759, 7.4759) [0.7514, 0.7513, 0.7513]
  PeriodicSite: Ca (2.4747, 2.4744, 7.4759) [0.2487, 0.2486, 0.7513]
  PeriodicSite: Ca (2.4747, 7.4759, 2.4744) [0.2487, 0.7513, 0.2486]
  PeriodicSite: Ca (2.4705, 7.4801, 7.4801) [0.2483, 0.7517, 0.7517]
  PeriodicSite: Fe (0.0067, 9.9439, 9.9439) [0.0007, 0.9993, 0.9993]
  PeriodicSite: Fe (4.9721, 4.9785, 9.9466) [0.4997, 0.5003, 0.9996]
  PeriodicSite: Fe (0.0039, 4.9785, 4.9785) [0.0004, 0.5003, 0.5003]
  PeriodicSite: Fe (4.9721, 9.9466, 4.9785) [0.4997, 0.9996, 0.5003]
  PeriodicSite: Co (4.9696, 9.9460, 9.9460) [0.4995, 0.9995, 0.9995]
  P

Now that the pba data is loaded into python, we can begin to building pymatgen entries for each structure.

## Using Pymatgen

In [11]:
import pymatgen as mg

### Creating pymatgen entries

Next, we want to make pymatgen entries using the composition and energy values. Here is an example of a ComputedEntry:

In [12]:
from pymatgen.entries.computed_entries import ComputedEntry

my_entry = ComputedEntry(composition="Ni4O2",
                  energy=-28,
                  parameters={"potcar_symbols": ['pbe Ni_pv', 'pbe O'],
                              "hubbards":{'Ni': 6.2, 'O': 0.0}},
                  data={"oxide_type":"oxide"})

print(my_entry)

ComputedEntry None - Ni4 O2
Energy = -28.0000
Correction = 0.0000
Parameters:
potcar_symbols = ['pbe Ni_pv', 'pbe O']
hubbards = {'Ni': 6.2, 'O': 0.0}
Data:
oxide_type = oxide


The first step to creating a ComputedEntry is gettting the composition, which can be given either as a dict or as a string.

In [13]:
struct=data_1[1]['input']['structure']

In [14]:
struct.composition

Comp: Ca4 Fe4 Co4 C24 N24

Next, we find how to access the energy value from the 'output' section of the main data_1 file:

In [15]:
out = data_1[1]['output']
out['energy']

-476.8670732

Now we can create a pymatgen entry with the pba structure in data_1[1]:

In [21]:
struct=data_1[1]['input']['structure']
pba_1 = ComputedEntry(composition=struct.composition,
                  energy=data_1[1]['output']['energy'],
                      parameters = {"nelect": data_1[1]['input']['parameters']['NELECT'],
                                    "hubbards": data_1[1]['input']['hubbards'],
                                    "potcar_spec": data_1[1]['input']['potcar_spec'],
                                    "is_hubbard": data_1[1]['input']['is_hubbard']})

print(pba_1)

ComputedEntry None - Ca4 Fe4 Co4 C24 N24
Energy = -476.8671
Correction = 0.0000
Parameters:
nelect = 348.0
hubbards = {}
potcar_spec = [{'titel': 'PAW_PBE Ca_sv 06Sep2000', 'hash': 'eb006721e214c04b3c13146e81b3a27d'}, {'titel': 'PAW_PBE Fe_pv 06Sep2000', 'hash': '5963411539032ec3298fa675a32c5e64'}, {'titel': 'PAW_PBE Co 06Sep2000', 'hash': 'b169bca4e137294d2ab3df8cbdd09083'}, {'titel': 'PAW_PBE C 08Apr2002', 'hash': 'c0a8167dbb174fe492a3db7f5006c0f8'}, {'titel': 'PAW_PBE N 08Apr2002', 'hash': 'b98fd027ddebc67da4063ff2cabbc04b'}]
is_hubbard = False
Data:


Shyam also said to add the 'run_type' to the parameters of each entry using the command d\["run_type"] =  "calcs_reversed.0.run_type". As far as I can tell, the pba files in the dictionary do not have any parameters for 'run_type' or 'calcs_reversed', so for now I'll just leave it out of the parameters.

Using Ctrl-f with run_type through the data_1[i] files did not return any results:

In [23]:
data_1[100]

{'_id': '58e66ecdd95cbb63a64888cb', 'input': {'structure': Structure Summary
  Lattice
      abc : 10.166 10.166 10.166
   angles : 90.0 90.0 90.0
   volume : 1050.6312542960002
        A : 10.166 0.0 0.0
        B : 0.0 10.166 0.0
        C : 0.0 0.0 10.166
  PeriodicSite: K (7.6245, 7.6245, 7.6245) [0.7500, 0.7500, 0.7500]
  PeriodicSite: K (2.5415, 2.5415, 7.6245) [0.2500, 0.2500, 0.7500]
  PeriodicSite: K (2.5415, 7.6245, 2.5415) [0.2500, 0.7500, 0.2500]
  PeriodicSite: K (7.6245, 2.5415, 2.5415) [0.7500, 0.2500, 0.2500]
  PeriodicSite: Zn (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
  PeriodicSite: Zn (5.0830, 5.0830, 0.0000) [0.5000, 0.5000, 0.0000]
  PeriodicSite: Zn (0.0000, 5.0830, 5.0830) [0.0000, 0.5000, 0.5000]
  PeriodicSite: Zn (5.0830, 0.0000, 5.0830) [0.5000, 0.0000, 0.5000]
  PeriodicSite: Fe (5.0830, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
  PeriodicSite: Fe (0.0000, 5.0830, 0.0000) [0.0000, 0.5000, 0.0000]
  PeriodicSite: Fe (0.0000, 0.0000, 5.0830) [0.0000, 0.

Now that we know how to create a pymatgen entry, let's iterate over the whole data_1 file with this method to make a list of pba entries:

In [24]:
pba_entries = []
for i in range(0, len(data_1)):
    if 'input' in data_1[i]:
        struct = data_1[i]['input']['structure']
        pba_entry = ComputedEntry(composition = struct.composition,
                                 energy = data_1[i]['output']['energy'],
                                 parameters = {"nelect": data_1[i]['input']['parameters']['NELECT'],
                                    "hubbards": data_1[i]['input']['hubbards'],
                                    "potcar_spec": data_1[i]['input']['potcar_spec'],
                                    "is_hubbard": data_1[i]['input']['is_hubbard']})
        pba_entries.append(pba_entry)
pba_entries[:2]

[ComputedEntry None - Ca4 Fe4 Co4 C24 N24
 Energy = -476.8671
 Correction = 0.0000
 Parameters:
 nelect = 348.0
 hubbards = {}
 potcar_spec = [{'titel': 'PAW_PBE Ca_sv 06Sep2000', 'hash': 'eb006721e214c04b3c13146e81b3a27d'}, {'titel': 'PAW_PBE Fe_pv 06Sep2000', 'hash': '5963411539032ec3298fa675a32c5e64'}, {'titel': 'PAW_PBE Co 06Sep2000', 'hash': 'b169bca4e137294d2ab3df8cbdd09083'}, {'titel': 'PAW_PBE C 08Apr2002', 'hash': 'c0a8167dbb174fe492a3db7f5006c0f8'}, {'titel': 'PAW_PBE N 08Apr2002', 'hash': 'b98fd027ddebc67da4063ff2cabbc04b'}]
 is_hubbard = False
 Data:, ComputedEntry None - Mg4 Cr4 Os4 C24 N24
 Energy = -501.1275
 Correction = 0.0000
 Parameters:
 nelect = 352.0
 hubbards = {}
 potcar_spec = [{'titel': 'PAW_PBE Mg_pv 06Sep2000', 'hash': 'bbcf6f81cc34a3090d483ad641178746'}, {'titel': 'PAW_PBE Cr_pv 07Sep2000', 'hash': 'eb23364cc25164418f9f79efd8f04f7d'}, {'titel': 'PAW_PBE Os_pv 20Jan2003', 'hash': '7bb96cace1809ebeb4d030d71024c5bf'}, {'titel': 'PAW_PBE C 08Apr2002', 'hash': '

This looks good so far.

### Accessing materials entries from the Materials Project

Next, we access the Materials Project's main entries list of structures made out of the given atoms.

In [44]:
#How to get a list of the component atoms from the pymatgen structure:
pba_a = pba_entries[0]

In [45]:
comp = pba_a.composition.as_dict().keys()
print(comp)

dict_keys(['Ca', 'Fe', 'Co', 'C', 'N'])


In [46]:
from pymatgen import MPRester
mpr = MPRester(api_key='clRGHmBDgp1xt9zA')

In [47]:
entries = mpr.get_entries_in_chemsys(comp)
print(entries[:2])

[ComputedEntry mp-10683 - Ca1
Energy = -1.6033
Correction = 0.0000
Parameters:
run_type = GGA
is_hubbard = False
pseudo_potential = {'functional': 'PBE', 'labels': ['Ca_sv'], 'pot_type': 'paw'}
hubbards = {}
potcar_symbols = ['PBE Ca_sv']
oxide_type = None
Data:
oxide_type = None, ComputedEntry mp-45 - Ca1
Energy = -2.0218
Correction = 0.0000
Parameters:
run_type = GGA
is_hubbard = False
pseudo_potential = {'functional': 'PBE', 'labels': ['Ca_sv'], 'pot_type': 'paw'}
hubbards = {}
potcar_symbols = ['PBE Ca_sv']
oxide_type = None
Data:
oxide_type = None]


entries now contains all of the Materials Project structures containing any combination of 'Ca', 'Fe', 'Co', 'C', and 'N'. Let's go ahead and add the pba with these atoms to this list:

In [48]:
entries.append(pba_entries[0])
print(entries[-1])

ComputedEntry None - Ca4 Fe4 Co4 C24 N24
Energy = -476.8671
Correction = 0.0000
Parameters:
nelect = 348.0
hubbards = {}
potcar_spec = [{'titel': 'PAW_PBE Ca_sv 06Sep2000', 'hash': 'eb006721e214c04b3c13146e81b3a27d'}, {'titel': 'PAW_PBE Fe_pv 06Sep2000', 'hash': '5963411539032ec3298fa675a32c5e64'}, {'titel': 'PAW_PBE Co 06Sep2000', 'hash': 'b169bca4e137294d2ab3df8cbdd09083'}, {'titel': 'PAW_PBE C 08Apr2002', 'hash': 'c0a8167dbb174fe492a3db7f5006c0f8'}, {'titel': 'PAW_PBE N 08Apr2002', 'hash': 'b98fd027ddebc67da4063ff2cabbc04b'}]
is_hubbard = False
Data:


### Applying Correction

Now that we have a list of pymatgen entries, including both MP entries and our pba, we apply corrections using the MaterialsProjectCompatibility module.

In [43]:
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
mpc = MaterialsProjectCompatibility()

In [50]:
corrected_entries = mpc.process_entries(entries)

In [53]:
print(corrected_entries[:2])

[ComputedEntry mp-10683 - Ca1
Energy = -1.6033
Correction = 0.0000
Parameters:
run_type = GGA
is_hubbard = False
pseudo_potential = {'functional': 'PBE', 'labels': ['Ca_sv'], 'pot_type': 'paw'}
hubbards = {}
potcar_symbols = ['PBE Ca_sv']
oxide_type = None
Data:
oxide_type = None, ComputedEntry mp-45 - Ca1
Energy = -2.0218
Correction = 0.0000
Parameters:
run_type = GGA
is_hubbard = False
pseudo_potential = {'functional': 'PBE', 'labels': ['Ca_sv'], 'pot_type': 'paw'}
hubbards = {}
potcar_symbols = ['PBE Ca_sv']
oxide_type = None
Data:
oxide_type = None]


Most of the entries have a correction value of 0, but there are a handful with nonzero correction values.

### Accessing Energy above Hull

Now that we have a list of corrected entries for the atoms in our first PBA, we can calculate a phase diagram for these atoms. This phase diagram is then used to calculate the energy above hull for the PBA.

In [54]:
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter

In [55]:
phase_d = PhaseDiagram(entries)

In [60]:
#Plotting the phase diagram:
try:
    plotter = PDPlotter(phase_d, show_unstable=True)
    plotter.show()
except Exception as e:
    print(e)
    print(type(e))

Only 1-4 components supported!
<class 'ValueError'>


PBAs have 5 different elements and PDPlotter can only plot diagrams with 4 or fewer elements, so we unfortunately are unable to visualize the phase diagram. However, we can still calculate the e_above_hull using the phase_d object.

In [59]:
#Calculating e_above_hull for the last object in entries, which is our PBA.
e_above_hull = phase_d.get_e_above_hull(entries[-1])
print(e_above_hull)

0.43556256118266745


### Looping through PBAs

Now that we know how to access all of the necessary data for each PBA, we can loop through the pba_entries structure to get data for every PBA. We will also collect the composition of each PBA. We will store this data in the pandas dataframe pba_e_hull_df.

Note that we only need to collect e_above_hull data for the PBA structures; we don't care about e_above_hull for the MP structures.

In [ ]:
# #First, initialize a dataframe filled with NaN, which we will fill as we go through pba_entries:
# pba_e_hull_df = pd.DataFrame(index=range(len(pba_entries)),columns=['Composition','e_above_hull','A_atom','P_atom','R_atom','Correction'])
# for i in range(len(pba_entries)): #Looping through the length of the pba_entries list
#     try:
#         entries = mpr.get_entries_in_chemsys(pba_entries[i].as_dict()['composition'].keys()) #access entries from MP
#         entries.append(pba_entries[i]) #adding pba to the end of the list of MP entries
#         entries = mpc.process_entries(entries) #applying correction
#         phase_d = PhaseDiagram(entries) #making phase diagram, which will allow calculation of e_above_hull
    
#         #Putting composition, e_above_hull, and correction values into pba_df:
#         pba_e_hull_df.loc[i, 'Composition'] = entries[-1].composition.formula
#         pba_e_hull_df.loc[i, 'e_above_hull'] = phase_d.get_e_above_hull(entries[-1])
#         pba_e_hull_df.loc[i, 'Correction'] = entries[-1].correction
    
#         if len(list(entries[-1].composition.as_dict())) == 4:
#             #This will be true when P and R are the same atom, in which case we need to index differently.
#             pba_e_hull_df.loc[i, 'A_atom'] = list(entries[-1].composition.as_dict())[0]
#             pba_e_hull_df.loc[i, 'P_atom'] = list(entries[-1].composition.as_dict())[1]
#             pba_e_hull_df.loc[i, 'R_atom'] = list(entries[-1].composition.as_dict())[1]
#         else:
#             pba_e_hull_df.loc[i, 'A_atom'] = list(entries[-1].composition.as_dict())[0]
#             pba_e_hull_df.loc[i, 'P_atom'] = list(entries[-1].composition.as_dict())[1]
#             pba_e_hull_df.loc[i, 'R_atom'] = list(entries[-1].composition.as_dict())[2]
#     except:
#         print('The error occured on loop ' + str(i))

Because we had to access the entries from the main MP database, this code took ~2 hours to run completely. Oddly, it also ran into some errors on several of the PBAs. 

When I looped the same code over those PBAs again, there were no errors (except for 1 structure) and I inserted the data into the pba_e_hull_df dataframe.

In [ ]:
# error_entries = [1,12,21,40,56,134,185,187,258,271,273,277,429,456,475,493]
# for i in error_entries:
#     try:
#         entries = mpr.get_entries_in_chemsys(pba_entries[i].as_dict()['composition'].keys()) #access entries from MP
#         entries.append(pba_entries[i])
#         entries = mpc.process_entries(entries) #applying correction
#         phase_d = PhaseDiagram(entries) #making phase diagram, which will allow calculation of e_above_hull
    
#         #Putting composition, e_above_hull, and correction values into pba_df:
#         pba_e_hull_df.loc[i, 'Composition'] = entries[-1].composition.formula
#         pba_e_hull_df.loc[i, 'e_above_hull'] = phase_d.get_e_above_hull(entries[-1])
#         pba_e_hull_df.loc[i, 'Correction'] = entries[-1].correction
    
#         if len(list(entries[-1].composition.as_dict())) == 4:
#             #This will be true when P and R are the same atom, in which case we need to index differently.
#             pba_e_hull_df.loc[i, 'A_atom'] = list(entries[-1].composition.as_dict())[0]
#             pba_e_hull_df.loc[i, 'P_atom'] = list(entries[-1].composition.as_dict())[1]
#             pba_e_hull_df.loc[i, 'R_atom'] = list(entries[-1].composition.as_dict())[1]
#         else:
#             pba_e_hull_df.loc[i, 'A_atom'] = list(entries[-1].composition.as_dict())[0]
#             pba_e_hull_df.loc[i, 'P_atom'] = list(entries[-1].composition.as_dict())[1]
#             pba_e_hull_df.loc[i, 'R_atom'] = list(entries[-1].composition.as_dict())[2]
#     except:
#         print('Error on loop ' + str(i))

In [ ]:
# #Saving to file as pba_e_hull_df.csv
# pba_e_hull_df.to_csv('pba_e_hull_df.csv')

The code ran into an error again on index 456 in pba_entries. The error occured when I was parsing its composition into the A, P, and R atoms. Let's examine this structure.

In [63]:
print(pba_entries[456])

ComputedEntry None - C8 N12
Energy = -141.5829
Correction = 0.0000
Parameters:
nelect = 92.0
hubbards = {}
potcar_spec = [{'titel': 'PAW_PBE C 08Apr2002', 'hash': 'c0a8167dbb174fe492a3db7f5006c0f8'}, {'titel': 'PAW_PBE N 08Apr2002', 'hash': 'b98fd027ddebc67da4063ff2cabbc04b'}]
is_hubbard = False
Data:


In [66]:
# #We can look at this data in the original data_1 structure as well:
# print(data_1[457])

This structure has chemical formula C8 N12, so it is obviously not a PBA. In the pba_e_hull_df, the row for that element is just NaN for the A, P, and R values, so we'll just leave it like that for now. 

I saved the pba_e_hull_df into a csv file so that we don't have to wait several hours for the code to run every time we restart the notebook. Instead, we just use the pandas read_csv function to load the file into a dataframe.

In [69]:
pba_e_hull_df = pd.read_csv('pba_e_hull_df.csv', sep = ',')

In [70]:
pba_e_hull_df.head()

,Unnamed: 0,Composition,e_above_hull,A_atom,P_atom,R_atom,Correction
0,0,Ca4 Fe4 Co4 C24 N24,0.435563,Ca,Fe,Co,0
1,1,Mg4 Cr4 Os4 C24 N24,0.319625,Mg,Cr,Os,0
2,2,Ca4 Mn4 Fe4 C24 N24,0.382620,Ca,Mn,Fe,0
3,3,Ca4 Mn4 Os4 C24 N24,0.303886,Ca,Mn,Os,0
4,4,Li4 Cr8 C24 N24,0.320146,Li,Cr,Cr,0
